# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **Classification. Its a binary classification problem as we need to predict whether a student need intervention or not based on whether the student would fail to graduate

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import display # Allows the use of display() for DataFrames
%matplotlib inline


# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"



Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data .shape[0]

# TODO: Calculate number of features
n_features = student_data .shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data [student_data ['passed'] == "yes"].shape[0]
#n_passed = df_passed.shape[0]

# TODO: Calculate failing students
n_failed = student_data [student_data ['passed'] == "no"].shape[0]
#n_passed = df_passed.shape[0]

# TODO: Calculate graduation rate
grad_rate = float(float(n_passed)/float(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)


Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

#print X_all

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print y_all.head()



Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [9]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

#print X_all


#pd.scatter_matrix(X_all, alpha = 0.3, figsize = (14,8), diagonal = 'kde')

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

test_size_percent = (float(num_test)/float(X_all.shape[0]))
# TODO: Shuffle and split the dataset into the number of training and testing points above

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=95, random_state=42)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])


Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** 
1. Random Forest:

a. Random Forest is mostly used in the field of Bioinformatics.Random Forest technique includes an ensemble of decision trees and incorporates feature selection and interactions naturally in the learning
process. It is efficient and has a high prediction accuracy for many types of data. Recent work in computational biology has shown an increased use of random forest, owing to its unique advantages
in dealing with small sample size, high-dimensional feature space, and complex data
structures.

Source: http://www.cs.cmu.edu/~qyj/papersA08/11-rfbook.pdf

b. Strengths:

- It runs efficiently on large data datasets 
- It can handle thousands of input variables without variable deletion.
- It gives estimates of what variables are important in the classification.
- fits a number of decision tree classifiers on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting. 
- Random Forest uses fully grown decision trees (low bias, high variance). It tackles the error reduction task by reducing variance. It mostly donot overfit.
- It offers an experimental method for detecting variable interactions.
- It has an effective method for estimating missing data and maintains accuracy when a large proportion of the data are missing
- RF is practicaly tuning free. We don't need to do any parameter tuning to find the optimal model

Source: https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
http://stats.stackexchange.com/questions/173390/gradient-boosting-tree-vs-random-forest

c. Weeknesses:

- Random forests have been observed to overfit for some datasets with noisy classification/regression tasks.
- For data including categorical variables with different number of levels, random forests are biased in favor of those attributes with more levels. Therefore, the variable importance scores from random forest are not reliable for this type of data.

Source: www.dabi.temple.edu/.../Montillo_RandomForests_4-2-2009.pdf


d. I think Random Forest is a good candidate for this problem - We have a high featured dataset and RF is efficient for that and also RF do not expect linear features or even features that interact linearly. RF is good at handling categorical features (Most of our features are categorical). Random Forest uses fully grown decision trees (low bias, high variance). It tackles the error reduction task in the opposite way: by reducing variance. This means RF is practically tuning free - no need of tuning parameters using cross validation.

Source: https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms
Source: http://stats.stackexchange.com/questions/173390/gradient-boosting-tree-vs-random-forest


2. Gradient Boosting:

a. As quoted in the paper in the link below: Gradient Boosting is mostly used in the field of genomic research. Gradient Boosting can be used as a SNP Filter - an Evaluation Using Simulated and Hair Morphology Data. SNP stands for single-nucleotide polymorphisms. Typically, genome-wide association studies consist of regressing the phenotype on each SNP separately using an additive genetic model. Although statistical models for recessive, dominant, SNP-SNP, or SNP-environment interactions exist, the testing burden makes an evaluation of all possible effects impractical for genome-wide data. A two-step approach is advocated where the first step consists of a filter that is sensitive to different types of SNP main and interactions effects. The aim is to substantially reduce the number of SNPs such that more specific modeling becomes feasible in a second step. “gradient boosting machine” (GBM) can be used as a filter. GBM builds a predictive model iteratively by adding “weak learners” (simple trees). The “weak learners” can be small regression or classification trees that are grown to a user- specified, usually small, number of splits.  At each split a single SNP is selected to split the sample into two parts called “daughter nodes”. 

Source: http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3882018/

b. Strengths:

- Boosting is based on weak learners (high bias, low variance). In terms of decision trees, weak learners simple trees trees, say for e.g trees with two leaves. Boosting reduces error mainly by reducing bias. The idea is to add a classifier at a time, so that the next classifiers is trained to improve the already trained ensemble.GB build trees one at a time, where each new tree helps to correct errors made by previously trained tree. With each tree added, the model becomes even more expressive. There are typically three parameters - number of trees, depth of trees and learning rate, and the each tree built is generally shallow. 

- GB training generally takes longer because of the fact that trees are built sequentially. However benchmark results have shown GBDT are better learners than Random Forests. 

- GB has most of strengths of RF and can used for similar use case except that GB needs hyper parameter with  tunes to avoid overfitting and its slow compared to RF as trees are built sequentially starting with a week learner correcting the errors made by the previous tree in the next tree.

- Research and experiments say that as a whole a well tuned GB can outperform an RF

Source: http://stats.stackexchange.com/questions/173390/gradient-boosting-tree-vs-random-forest
https://www.quora.com/When-would-one-use-Random-Forests-over-Gradient-Boosted-Machines-GBMs
https://www.quora.com/What-are-the-advantages-disadvantages-of-using-Gradient-Boosting-over-Random-Forests

c. Weeknesses:

- GB could be very slow. Gradient Boosted Machines improves the learner in every iteration squentially. So, if we are constrained by the size of the data GB might not be the best option.

- GB are prone to overfitting. We must tune the hyperparameters to find the best model


Source: http://stats.stackexchange.com/questions/173390/gradient-boosting-tree-vs-random-forest
https://www.quora.com/When-would-one-use-Random-Forests-over-Gradient-Boosted-Machines-GBMs
https://www.quora.com/What-are-the-advantages-disadvantages-of-using-Gradient-Boosting-over-Random-Forests


d. Most ensembled decision tree methods do not expect linear features or even features that interact linearly. Tree Ensembles, because they are nothing more than a bunch of Decision Trees, can handle this categorical features well. These algorithms handle very well high dimensional spaces as well as large number of training examples. As our dataset is high dimensional as well as categorical an ensembled decision tree model like GB suits well. 


3. Stochastic Gradient Descent (SGD):

a.
* Here I have provided refrence of a research work where Stochastic Gradient has used on a regression problem and not on a classification problem

Stochastic Gradient Descent Algorithm for Performance Contribution of Employees- To identify the contribution of each employees belonging to each department towards the overall performance of a company is very challenging. There are not many researches done on application of data driven approach like machine learning to overcome this challenge. The paper quoted below describes adaptability of stochastic gradient descent algorithm in evaluating the performance contribution. 
In the experiment quoted in this paper, Stochastic gradient descent algorithm is used to find the weights(coefficients) for performance contributions for the linear regression equation. The paper mentions that more localized optimal solution has been attained using SGD to determine the determine the weights(compared to vanilla gradient descent). SGD does micro batch updates on the parameters to minimize the error by running small samples of data compared to GD which only update the weights after running through all the data points. So SGD would converge faster for large datasets and SGD starts improving itself right away from the first sample.

Source: http://iosrjournals.org/iosr-jbm/papers/Vol16-issue6/Version-3/I016637780.pdf
https://www.quora.com/Whats-the-difference-between-gradient-descent-and-stochastic-gradient-descent


b. Strengths:

Both gradient descent (GD) and stochastic gradient descent (SGD) update Set of parameters to minimize an error function. Gradient Descent needs to run on all the training data samples for a parameter update in a particular iteration, SGD, uses a training sample from training set to do the update for a parameter in a particular iteration.

If the training dataset is quite huge, GD may take too long to update the parameters and convergence would be slow
to update the values of the parameters as we are running through the complete training set. On the other hand, SGD will be faster as we can do micro batch update on the parameters after running through each micro sample of training set.S o Stochastic Gradient Descent will converge much faster compared to vanilla Gradient Descent.

Source: https://www.quora.com/Whats-the-difference-between-gradient-descent-and-stochastic-gradient-descent
Source: Udacity forum - https://discussions.udacity.com/t/gradient-descent-issues-quiz/177769

c. Weeknesses:

- SGD often converges much faster compared to GD but the error function is not as well minimized as in the case of GD. The cost function in GD is deterministic but SGD takes a zig zag by stepping through different samples of data and final reach the globla minima. Due to its stochastic nature, SGD's path towards the global cost minimum is not direct as in GD

Source: https://www.quora.com/Whats-the-difference-between-gradient-descent-and-stochastic-gradient-descent
 
 - Another weekness for SGD would be in using constant step sizes .It may not guarantee convergence to the global optimum. We might oscillate around near the global optima without actually converging if we donot keep decreasing the step sizes as we keep iterating. The main disadvantage of using a constant step size is that if the function is not smooth enough, we might overshoot and end up at a higher function value than where we were previously.
 
https://www.quora.com/What-are-the-disadvantages-of-having-a-constant-step-size-for-batch-Gradient-Descent

d. For large datasets as well as datasets with many dimensions SVM right out of the box might not perform well as training time would be quadratic. Also, Gradient Descent might take lot of time to converge if the dataset is large or has too many dimensions. Stochastic Gradient Descent Classifier (with SVM) (prerebaly with a non-linear SVM as there are many dimensions chances are that their will be a non-linear margin between the hyperplanes - loss=‘squared_hinge’ ) might be better off in this case because of batch updates and might converge faster compared to a Gradient descent based classifier.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    #return f1_score(target.values, y_pred, pos_label='yes')
    #return f1_score(target.values, y_pred, pos_label=None, average='weighted')
    return f1_score(target.values, y_pred, pos_label=None, average='weighted')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state = 1)
clf_B = GradientBoostingClassifier(random_state = 1)
clf_C = linear_model.SGDClassifier(loss='squared_hinge', random_state = 1)


# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

for size in [100, 200, 300]:
    train_predict(clf_A, X_train[:size], y_train[:size], X_test, y_test)
    
for size in [100, 200, 300]:
    train_predict(clf_B, X_train[:size], y_train[:size], X_test, y_test)
    
for size in [100, 200, 300]:
    train_predict(clf_C, X_train[:size], y_train[:size], X_test, y_test)



Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0287 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.9597.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.6610.
Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0237 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.6018.
Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0251 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 0.9933.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.6659.
Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.0490 seconds
Made predictions in 0.0026 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.6300.
Training a GradientBoostingC

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - clf_A - RandomForestClassifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  .0192                       |0.0007                        | .9846                 | .6822    |
| 200               |    .0167                     |     0.0009                   |  .9744                |   .7519              |
| 300               |   0.0176                      |     0.0008                   |  .9976                |  .7313       |

** Classifer 2 - clf_B - GradientBoostingClassifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0392                     |   0.0003                      |1                  |  .7519               |
| 200               |    0.0533              |   0.0003                     |  0.9964                | 0.7591                |
| 300               |   0.0684                      |    0.0028                    |  0.9739                | 0.7794        |

** Classifer 3 - clf_C - linear_model.SGDClassifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0005                      |    0.0001                    | 0.7097                 | 0.6942                
| 200               |   0.0005                      |    0.0001                 |     0.8440             |   0.7682              
| 300               |    0.0006                     |      0.0002                  | 0.8226                 | 0.7763                |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** It looks like GradientBoostingClassifier based model would be the best model to use on the student data. Performancewise this model is the best with a F1 score of .7794 on the test dataset with the model trained on all 300 training points. Also the F1 score on the training data is .9739 which is quite high (not a highly biased model at all) and a score of .7794 indicates there is overfitting but this is the best we have achieved among all three models. We can definitely  increase the F1 score on the test data by tuning the model as GradientBoosting requires hyperparameter tuning to reduce overfitting. The training and testing time are well below 1 sec (.068 sec and .0028 sec) and would use minimum resources. 

Basically decision tree based models are compute heavy - in terms of BIG O its quadratic but in our case we dont have a very big dataset (300 records and 30 dimensions only) and the training and testing time are well under one sec. Based on the limited resource and cost we can safely accept GradientBoostingClassifier as the best model among the three models as its giving us the best performance.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** Gradient Boosting is a combination of multiple Decision based models. It starts with a simple model for training. Boosting reduces error mainly by increasing the robustness of the next model . The idea is to add a model at a time, so that the next model is trained to improve the already trained combination of models. So, the combined/ensembled becomes more expressive. The training phase takes little longer compared to other models because of the fact that the models are built sequentially. However in terms prediction accuracy, research shows Gradient Boosting is better then just "simple Decision tree" as well other "combined decision tree" based models such as Random Forest. We can also see that prediction accuracy of the model on the test dataset has increased from 75% to 78% when we trained the model on 300 datapoints than on 100 data points.

Based on the decision boundary established during the training phase, the model would evaluate a new record and identify the boundary as Pass (yes) or fail (no). For e.g. A student who does not fulltime job, less time, whose travel parents went to college, have access to internet is likely to fall in the "pass" bounday.

*** Note: Its impossible to vizualize a decision surface which is based on many different features ( 30 in this case) which is a hyerplane. Vizualizing a decision surface based on only two different features would not be right in this case as there too many features that are affecting hyperplane decision boundary. My explanation on prediction in Layman terms is purely based on some observations as well best guess jugement call on some features. I am finding it hard to imagine a complex decision tree based on so many features.

Source: http://stats.stackexchange.com/questions/173390/gradient-boosting-tree-vs-random-forest
https://www.quora.com/When-would-one-use-Random-Forests-over-Gradient-Boosted-Machines-GBMs
https://www.quora.com/What-are-the-advantages-disadvantages-of-using-Gradient-Boosting-over-Random-Forests


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [9]:
#number of trees, depth of trees and learning rate

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
# TODO: Import 'GridSearchCV' and 'make_scorer'


start = time()
    

# TODO: Create the parameters list you wish to tune
parameters = {'loss' : ['deviance','exponential'], 'learning_rate' : [.004, .005], 'n_estimators' : [100,200,300],
             'max_depth' : range(1, 11)}

# TODO: Initialize the classifier
clf = clf_B

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters)
                        #, f1_scorer)
  
print "GridSearchCV"

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

end = time()  

print "Grid Search on Classifier B is done in {:.4f} seconds".format(end - start)

# Get the estimator
clf = grid_obj.best_estimator_


    
    # Print the results


# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))


GridSearchCV
Grid Search on Classifier B is done in 94.1969 seconds
Made predictions in 0.0015 seconds.
Tuned model has a training F1 score of 0.9031.
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.7891.


In [132]:
#print clf.feature_importances_
#print clf.estimators_

In [10]:
#get parameters 
params = clf.get_params()
print params

import json

with open('/Users/212452403/Desktop/Udacity_Machine_Learning_Nano_degree/student_intervention/params.json', 'w') as outfile:
    json.dump(clf.get_params(), outfile)

{'presort': 'auto', 'loss': 'deviance', 'verbose': 0, 'subsample': 1.0, 'max_leaf_nodes': None, 'learning_rate': 0.004, 'warm_start': False, 'min_samples_leaf': 1, 'n_estimators': 100, 'min_samples_split': 2, 'init': None, 'min_weight_fraction_leaf': 0.0, 'random_state': 1, 'max_features': None, 'max_depth': 7}


In [11]:
with open('/Users/212452403/Desktop/Udacity_Machine_Learning_Nano_degree/student_intervention/params.json', 'r') as infile:
    print(infile.read())

{"presort": "auto", "loss": "deviance", "verbose": 0, "subsample": 1.0, "max_leaf_nodes": null, "learning_rate": 0.004, "warm_start": false, "min_samples_leaf": 1, "n_estimators": 100, "min_samples_split": 2, "init": null, "min_weight_fraction_leaf": 0.0, "random_state": 1, "max_features": null, "max_depth": 7}


In [38]:
#print dir(clf)

attrs_new = [i for i in dir(clf)]
print attrs_new

attrs =[i for i in dir(clf) if i.endswith('_') and not i.endswith('__')]
print attrs

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__getitem__', '__hash__', '__init__', '__iter__', '__len__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'alpha', 'apply', 'classes_', 'decision_function', 'estimators_', 'feature_importances_', 'fit', 'fit_transform', 'get_params', 'init', 'init_', 'learning_rate', 'loss', 'loss_', 'max_depth', 'max_features', 'max_features_', 'max_leaf_nodes', 'min_samples_leaf', 'min_sampl

In [40]:
attr_dict = {i: getattr(clf, i) for i in attrs}
attr_dict_new = {i: getattr(clf, i) for i in attrs_new}
#print attr_dict

In [47]:
#numpy array to list

import numpy as np 

for k in attr_dict:
    if isinstance(attr_dict[k], np.ndarray):
        attr_dict[k] = attr_dict[k].tolist()

#attr_dict_new_new = {}        
        
for k in attr_dict_new:
    if isinstance(attr_dict_new[k], np.ndarray):
        attr_dict_new[k] = attr_dict_new[k].tolist()

print attr_dict_new
        

{'__module__': 'sklearn.ensemble.gradient_boosting', '__abstractmethods__': frozenset([]), 'transform': <bound method GradientBoostingClassifier.transform of GradientBoostingClassifier(init=None, learning_rate=0.004, loss='deviance',
              max_depth=7, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=1, subsample=1.0, verbose=0,
              warm_start=False)>, '_is_initialized': <bound method GradientBoostingClassifier._is_initialized of GradientBoostingClassifier(init=None, learning_rate=0.004, loss='deviance',
              max_depth=7, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=1, subsample=1.0, verbose=0,
              warm_start=False)>, 'predict': <bound method Gradie

In [165]:
#print attr_dict

In [39]:
# Now dump the attribute dictionary to a JSON file

with open('/Users/212452403/Desktop/Udacity_Machine_Learning_Nano_degree/student_intervention/attributes.json','w') as outfile:
    #json.dump(attr_dict, outfile)
    print "nothing"
    


nothing


In [49]:
#print attr_dict
clf_new = GradientBoostingClassifier()
clf_new.set_params(**params)
#print clf_new.get_params()
#print attr_dict

### ['classes_', 'estimators_', 'feature_importances_', 'init_', 'loss_', 'max_features_', 'n_classes_', 'train_score_']

for k in attr_dict_new:
    #print k
    if isinstance(attr_dict_new[k], list):
        #setattr(clf_new, k, np.array(attr_dict[k]))
        #setattr(x, 'foobar', 123) is the same as x.foobar = 123.
        #if k == 'feature_importances_':
            #print attr_dict[k]
        clf_new.k = np.array(attr_dict_new[k])
        #elif k == 'estimators_':
            #clf_new.k = np.array(attr_dict[k])
        #elif k == 'init_':
            #clf_new.k = np.array(attr_dict[k])
            
        #if k == 'estimators_':
            #clf_new.k = np.array(attr_dict[k])
            
            

        #clf_new.__setattr__(k, np.array(attr_dict[k]))
    #else:
        #clf_new.k = attr_dict[k]
        #setattr(clf_new, k, attr_dict[k])
        #clf_new.__setattr__(k, np.attr_dict[k])
#clf_new.fit()




In [62]:
for k in attr_dict_new:
    print k
    print clf_new.k
#print clf_new.__init__

__module__
[ 1.24321862  1.23776774  1.23237201  1.22703231  1.22174524  1.21651168
  1.21132884  1.20619701  1.20111625  1.19608333  1.19108873  1.18615012
  1.18126129  1.17641519  1.17160175  1.16684774  1.16215137  1.15747043
  1.1528591   1.14827311  1.14372399  1.13920364  1.1347563   1.13034468
  1.12595567  1.12158758  1.11729332  1.11302045  1.1087718   1.10455184
  1.10038371  1.09608397  1.09177808  1.08745278  1.08319956  1.07894205
  1.07474624  1.07057301  1.0664272   1.06229277  1.05820182  1.05417372
  1.05014825  1.04617443  1.04220052  1.03826475  1.03439061  1.03050588
  1.026688    1.02289278  1.01910001  1.01533233  1.01170356  1.00798843
  1.0043828   1.00075093  0.99734736  0.99390966  0.99053799  0.98711638
  0.98382601  0.98045876  0.97721975  0.9739107   0.97068164  0.96746871
  0.96411457  0.96092113  0.95760496  0.9544827   0.95127565  0.94816735
  0.94493176  0.94186502  0.93867601  0.93564932  0.93255314  0.92950531
  0.92654078  0.9234463   0.92051978  0.

In [66]:
#print clf_new.get_params()
#pred = clf_new.predict(X_train)

print clf_new.__abstractmethods__

frozenset([])


In [63]:
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf_new, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf_new, X_test, y_test))

'''

Made predictions in 0.0015 seconds.
Tuned model has a training F1 score of 0.9031.
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.7891.
'''

NotFittedError: Estimator not fitted, call `fit` before making predictions`.

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final F1 score on training data is .8299 compared to 0.9739, test score is .8000 compared to .7794. This means the model is performing well on the test data although the performance on the train data has decreased, the model has definitely reached a fine tuned spot between bias and variance with optimal performace. The final model's variance has decreased compared to the previous one with little compromise on bias.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.